# Importing functions

In [40]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import joblib
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, create_time_series_features
from preprocess_time_series_features import preprocess_features_time_series, create_fast_preprocessing_ts, preprocess_features_time_series_odds, create_fast_preprocessing_ts_odds
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

today_date = '2023-12-10'

In [41]:
def preprocess_features_time_series_odds_preds(df_Serie_A, num_features, today_date):

    all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
    less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
    few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

    Train_df = df_Serie_A.iloc[:10]
    Valid_df = df_Serie_A.iloc[:10]
    Test_df = df_Serie_A[df_Serie_A['date']==today_date]

    Train_labels = Train_df[['ft_result']]
    Valid_labels = Valid_df[['ft_result']]
    Test_labels = Test_df[['ft_result']]
    
    Train_odds = Train_df[['home_win_odds','draw_odds','away_win_odds']]
    Valid_odds = Valid_df[['home_win_odds','draw_odds','away_win_odds']]
    Test_odds = Test_df[['home_win_odds','draw_odds','away_win_odds']]

    # preprocess Train dataframe
    Train_teams = Train_df[['stagione','hometeam','awayteam']]

    if num_features == 'all':
        features = all_features
        print('utilizzando tutte le features')
    elif num_features == 'less':
        print('utilizzando meno features')
        features = less_features
    else:
        print('utilizzando poche features')
        features=few_features

    Train_dict_features={}

    for feature in features:
        Train_dict_features[feature] = pd.DataFrame({})
        for colonna in Train_df.columns:
            pattern = re.compile(rf'^home_{feature}_\d+$')
            if pattern.match(colonna):
                Train_dict_features[feature][colonna]=Train_df[colonna]
        for colonna in Train_df.columns:
            pattern = re.compile(rf'^away_{feature}_\d+$')
            if pattern.match(colonna):
                Train_dict_features[feature][colonna]=Train_df[colonna]

    #preprocess valid dataframe
    Valid_teams = Valid_df[['stagione','hometeam','awayteam']]


    if num_features == 'all':
        features = all_features
        print('utilizzando tutte le features')
    elif num_features == 'less':
        print('utilizzando meno features')
        features = less_features
    else:
        print('utilizzando poche features')
        features=few_features

    Valid_dict_features={}

    for feature in features:
        Valid_dict_features[feature] = pd.DataFrame({})
        for colonna in Valid_df.columns:
            pattern = re.compile(rf'^home_{feature}_\d+$')
            if pattern.match(colonna):
                Valid_dict_features[feature][colonna]=Valid_df[colonna]
        for colonna in Valid_df.columns:
            pattern = re.compile(rf'^away_{feature}_\d+$')
            if pattern.match(colonna):
                Valid_dict_features[feature][colonna]=Valid_df[colonna]

    # preprocess test dataframe
    Test_teams = Test_df[['stagione','hometeam','awayteam']]

    if num_features == 'all':
        features = all_features
        print('utilizzando tutte le features')
    elif num_features == 'less':
        print('utilizzando meno features')
        features = less_features
    else:
        print('utilizzando poche features')
        features=few_features

    Test_dict_features={}

    for feature in features:
        Test_dict_features[feature] = pd.DataFrame({})
        for colonna in Test_df.columns:
            pattern = re.compile(rf'^home_{feature}_\d+$')
            if pattern.match(colonna):
                Test_dict_features[feature][colonna]=Test_df[colonna]
        for colonna in Test_df.columns:
            pattern = re.compile(rf'^away_{feature}_\d+$')
            if pattern.match(colonna):
                Test_dict_features[feature][colonna]=Test_df[colonna]

    #encoding teams
    # load the  transformer
    teams_transf = joblib.load('transformers/teams_transformer.pkl')

    Train_teams_encoded = teams_transf.transform(Train_teams)
    Valid_teams_encoded = teams_transf.transform(Valid_teams)
    Test_teams_encoded = teams_transf.transform(Test_teams)

    #encoding labels
    # load the  transformer
    ordinal_encoder = joblib.load('transformers/ordinal_encoder_transformer.pkl')

    Train_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Train_labels).reshape(-1, 1)))
    Valid_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Valid_labels).reshape(-1, 1)))
    Test_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Test_labels).reshape(-1, 1)))  

    #encoding numerical features
    Train_dict_features_norm = Train_dict_features.copy()
    Valid_dict_features_norm = Valid_dict_features.copy()
    Test_dict_features_norm = Test_dict_features.copy()

    for feature in list(Train_dict_features.keys()):
        # load the  transformer
        numerical_transf = joblib.load(f'transformers/numerical_{feature}_transformer.pkl')

        Train_dict_features_norm[feature] = numerical_transf.transform(Train_dict_features_norm[feature])
        Valid_dict_features_norm[feature] = numerical_transf.transform(Valid_dict_features_norm[feature])
        Test_dict_features_norm[feature] = numerical_transf.transform(Test_dict_features_norm[feature])
    
    # Encoding odds
    # load the  transformer
    odds_transf = joblib.load('transformers/odds_transformer.pkl')

    Train_odds_norm = odds_transf.transform(Train_odds)
    Valid_odds_norm = odds_transf.transform(Valid_odds)
    Test_odds_norm = odds_transf.transform(Test_odds)

    return (Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
            Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, Test_labels, 
            Train_dict_features, Valid_dict_features, Test_dict_features, Train_df, Valid_df, Test_df, 
            Train_odds_norm, Valid_odds_norm, Test_odds_norm)

# Obtaining the new odds

In [42]:
import re

# Definisci il pattern regex
pattern = r'>\d\d?\.\d\d<'

# Lista per contenere i risultati estratti
matches = []

# Leggi il file TXT
with open('new_odds.txt', 'r') as file:
    # Leggi il contenuto del file riga per riga
    for line in file:
        # Trova tutti i match nel formato >\d.\d{2,3}<
        found_matches = re.findall(pattern, line)
        
        # Estrai solo la parte \d.\d{2,3} e convertila in float
        extracted_values = [float(re.search(r'\d\.\d{2,3}', match).group()) for match in found_matches]
        
        # Aggiungi i risultati alla lista principale
        matches.extend(extracted_values)

# Organizza i risultati in liste lunghe 3
result_lists = [matches[i:i+3] for i in range(0, len(matches), 3)]

# Stampare i risultati
for i in range(len(result_lists)):
    print(f"new_rows.loc[{i},['B365H', 'B365D', 'B365A']] = {result_lists[-(i+1)]}")

new_rows.loc[0,['B365H', 'B365D', 'B365A']] = [4.34, 3.39, 1.87]
new_rows.loc[1,['B365H', 'B365D', 'B365A']] = [2.49, 3.18, 2.92]
new_rows.loc[2,['B365H', 'B365D', 'B365A']] = [1.22, 6.44, 2.67]
new_rows.loc[3,['B365H', 'B365D', 'B365A']] = [3.69, 3.25, 2.07]
new_rows.loc[4,['B365H', 'B365D', 'B365A']] = [2.15, 3.11, 3.66]


# Adding the new results to the dataframe

In [43]:
# Leggi il file CSV
df = pd.read_csv(r"C:\Users\Hp\Documents\Serie_A_dump\csv_predictions\december_four.csv", parse_dates=['Date'], dayfirst=True)

# Crea un DataFrame con 10 righe di zeri
new_rows = pd.DataFrame(0, index=range(10), columns=df.columns)

# Assegna valori specifici alle colonne 'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam' per ciascuna riga
new_rows.loc[0, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Juventus', 'Napoli',0,0,'D']
new_rows.loc[1, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Verona', 'Lazio',0,0,'D']
new_rows.loc[2, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Atalanta', 'Milan',0,0,'D']
new_rows.loc[3, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Inter', 'Udinese',0,0,'D']
new_rows.loc[4, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Frosinone', 'Torino',0,0,'D']
new_rows.loc[5, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Monza', 'Genoa',0,0,'D']
new_rows.loc[6, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Salernitana', 'Bologna',0,0,'D']
new_rows.loc[7, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Roma', 'Fiorentina',0,0,'D']
new_rows.loc[8, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Empoli', 'Lecce',0,0,'D']
new_rows.loc[9, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] = ['I1', f'{today_date}', '17:30', 'Cagliari', 'Sassuolo',0,0,'D']

new_rows.loc[0,['B365H', 'B365D', 'B365A']] = [2.23,3.20,3.37]
new_rows.loc[1,['B365H', 'B365D', 'B365A']] = [4.34, 3.39, 1.87]
new_rows.loc[2,['B365H', 'B365D', 'B365A']] = [2.49, 3.18, 2.92]
new_rows.loc[3,['B365H', 'B365D', 'B365A']] = [1.22, 6.44, 2.67]
new_rows.loc[4,['B365H', 'B365D', 'B365A']] = [3.69, 3.25, 2.07]
new_rows.loc[5,['B365H', 'B365D', 'B365A']] = [2.15, 3.11, 3.66]
new_rows.loc[6,['B365H', 'B365D', 'B365A']] = [4.50,3.45,1.85]
new_rows.loc[7,['B365H', 'B365D', 'B365A']] = [2.05,3.25,4.00]
new_rows.loc[8,['B365H', 'B365D', 'B365A']] = [2.50,3.15,2.95]
new_rows.loc[9,['B365H', 'B365D', 'B365A']] = [2.3,3.50,3]

# Aggiungi le nuove righe al DataFrame esistente
new_csv = pd.concat([df, new_rows], ignore_index=True)
new_csv['Date'] = pd.to_datetime(new_csv['Date'], format='%Y-%m-%d')

# Salva il DataFrame aggiornato su un nuovo file CSV
new_csv.to_csv(r"C:\Users\Hp\Serie_A\csv_predictions\december_four.csv")

len(new_csv['HomeTeam'].unique()), len(new_csv['AwayTeam'].unique())


(20, 20)

In [44]:
#preprocess the data
df_giornate = preprocess_match_days(r"C:\Users\Hp\Serie_A\csv_predictions")
num_features = 'less'
num_giornate = 4
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = create_time_series_features(num_features, Statistiche_squadre_dict, df_giornate, num_giornate).dropna()

(Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, 
    Test_labels, Train_dict_features, Valid_dict_features, Test_dict_features, Train_df, Valid_df, Test_df, 
    Train_odds_norm, Valid_odds_norm, Test_odds_norm) = preprocess_features_time_series_odds_preds(df_Serie_A, num_features, today_date)

feature_input_shape = Test_dict_features_norm[list(Test_dict_features_norm.keys())[0]].shape[1]
Train_teams_shape = Test_teams_encoded.shape[1]

Dataset_train_norm, Dataset_valid_norm, Dataset_test_norm = create_fast_preprocessing_ts_odds(Train_teams_encoded, Train_dict_features_norm, Train_labels_encoded,
                                                                                         Valid_teams_encoded, Valid_dict_features_norm,
                                                                    Valid_labels_encoded,Test_teams_encoded, Test_dict_features_norm, Test_labels_encoded, 
                                                                    Train_odds_norm, Valid_odds_norm, Test_odds_norm)

Reading file: december_four.csv
Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!
utilizzando meno features
preprocess finished
utilizzando meno features
utilizzando meno features
utilizzando meno features


In [45]:
#import the model 
odds_model = tf.keras.models.load_model(r'c:\Users\Hp\Serie_A\model_experiments\model_odds_time_series')


In [76]:
## Visualizziamo un po' di risultati 
model_odds_new_pred_probs = odds_model.predict((Dataset_test_norm))
model_odds_new_prob = model_odds_new_pred_probs.max(axis=1)
model_odds_new_predictions = model_odds_new_pred_probs.argmax(axis=1)
model_odds_new_compare = pd.DataFrame({
                                'stagione': list( Test_df['stagione'] ),
                                'hometeam': list( Test_df['hometeam'] ),
                                'awayteam': list( Test_df['awayteam'] ),
                                'preds': model_odds_new_predictions, 
                                'best_pred_prob': model_odds_new_prob,
                                })

model_odds_new_compare[['model_away_prob','model_draw_prob','model_home_prob']] = model_odds_new_pred_probs
model_odds_new_compare[['home_win_odds', 'draw_odds', 'away_win_odds']] = Test_df[['home_win_odds', 'draw_odds', 'away_win_odds']].values
model_odds_new_compare['snai_pred'] = np.argmin(Test_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)
model_odds_new_compare['snai_prob'] = np.nanmin(Test_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)

# Assegno ai valori encoded dei valori più comprensibili per vittoria pareggio sconfitta
conditions = [
(model_odds_new_compare['preds'] == 2),  # Condizione per Home Win
(model_odds_new_compare['preds'] == 0),  # Condizione per Away Win
(model_odds_new_compare['preds'] == 1)   # Condizione per draw
]
conditions_snai = [
(model_odds_new_compare['snai_pred'] == 0),  # Condizione per Home Win
(model_odds_new_compare['snai_pred'] == 2),  # Condizione per Away Win
(model_odds_new_compare['snai_pred'] == 1)   # Condizione per Draw
]
# Valori corrispondenti alle condizioni
values = ['H', 'A', 'D']

# Creazione della nuova colonna 'result' e 'points
model_odds_new_compare['preds'] = np.select(conditions, values)
model_odds_new_compare['snai_pred'] = np.select(conditions_snai, values)
model_odds_new_compare['money_won'] = model_odds_new_compare['best_pred_prob']*model_odds_new_compare['snai_prob']

# creo la colonna money won 
model_odds_new_compare['pred_odds'] = model_odds_new_compare.apply(lambda row: row['home_win_odds'] if row['preds'] == 'H' else (row['draw_odds'] if row['preds'] == 'D' 
                                                                                             else row['away_win_odds']), axis=1)
# Inserisci la colonna nella nuova posizione
insert_data = model_odds_new_compare['money_won']
model_odds_new_compare.drop(columns=['money_won'], inplace=True)
model_odds_new_compare.insert(5, 'money_won', insert_data)

1/1 [==============================] - 0s 20ms/step


In [77]:
model_odds_new_compare

,stagione,hometeam,awayteam,preds,best_pred_prob,money_won,model_away_prob,model_draw_prob,model_home_prob,home_win_odds,draw_odds,away_win_odds,snai_pred,snai_prob,pred_odds
0,2023/2024,Monza,Genoa,H,0.468950,1.008243,0.197100,0.333950,0.468950,2.15,3.11,3.66,H,2.15,2.15
1,2023/2024,Juventus,Napoli,H,0.515699,1.150008,0.222758,0.261544,0.515699,2.23,3.20,3.37,H,2.23,2.23
2,2023/2024,Roma,Fiorentina,H,0.571728,1.172041,0.160774,0.267498,0.571728,2.05,3.25,4.00,H,2.05,2.05
3,2023/2024,Inter,Udinese,H,0.646968,0.789301,0.125683,0.227349,0.646968,1.22,6.44,2.67,H,1.22,1.22
4,2023/2024,Salernitana,Bologna,A,0.483513,0.894499,0.483513,0.319884,0.196602,4.50,3.45,1.85,A,1.85,1.85
5,2023/2024,Empoli,Lecce,H,0.444262,1.110655,0.268530,0.287208,0.444262,2.50,3.15,2.95,H,2.50,2.50
6,2023/2024,Cagliari,Sassuolo,H,0.380544,0.875251,0.324276,0.295180,0.380544,2.30,3.50,3.00,H,2.30,2.30
7,2023/2024,Atalanta,Milan,A,0.344929,0.858873,0.344929,0.319587,0.335484,2.49,3.18,2.92,H,2.49,2.92
8,2023/2024,Frosinone,Torino,A,0.440713,0.912276,0.440713,0.340461,0.218826,3.69,3.25,2.07,A,2.07,2.07
9,2023/2024,Verona,Lazio,A,0.672901,1.258324,0.672901,0.220178,0.106921,4.34,3.39,1.87,A,1.87,1.87


In [79]:
Test_df

,div,giornata,stagione,date,hometeam,awayteam,home_ft_goals_1,home_ft_goals_2,home_ft_goals_3,home_ft_goals_4,home_ft_goals_conceded_1,home_ft_goals_conceded_2,home_ft_goals_conceded_3,home_ft_goals_conceded_4,home_shots_1,home_shots_2,home_shots_3,home_shots_4,home_fouls_done_1,home_fouls_done_2,home_fouls_done_3,home_fouls_done_4,home_corners_obtained_1,home_corners_obtained_2,home_corners_obtained_3,home_corners_obtained_4,home_reds_1,home_reds_2,home_reds_3,home_reds_4,away_ft_goals_1,away_ft_goals_2,away_ft_goals_3,away_ft_goals_4,away_ft_goals_conceded_1,away_ft_goals_conceded_2,away_ft_goals_conceded_3,away_ft_goals_conceded_4,away_shots_1,away_shots_2,away_shots_3,away_shots_4,away_fouls_done_1,away_fouls_done_2,away_fouls_done_3,away_fouls_done_4,away_corners_obtained_1,away_corners_obtained_2,away_corners_obtained_3,away_corners_obtained_4,away_reds_1,away_reds_2,away_reds_3,away_reds_4,ft_result,home_win_odds,draw_odds,away_win_odds
6986,I1,15,2023/2024,2023-12-10,Monza,Genoa,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,16.0,12.0,12.0,12.0,12.0,15.0,11.0,12.0,7.0,11.0,3.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,2.0,21.0,13.0,16.0,8.0,15.0,14.0,9.0,11.0,5.0,2.0,6.0,1.0,0.0,0.0,0.0,0.0,D,2.15,3.11,3.66
6985,I1,15,2023/2024,2023-12-10,Juventus,Napoli,2.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,18.0,4.0,18.0,4.0,8.0,12.0,14.0,21.0,3.0,1.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,3.0,1.0,1.0,0.0,15.0,11.0,18.0,21.0,2.0,9.0,11.0,10.0,6.0,4.0,8.0,9.0,0.0,0.0,0.0,0.0,D,2.23,3.20,3.37
6987,I1,15,2023/2024,2023-12-10,Roma,Fiorentina,2.0,3.0,0.0,2.0,1.0,1.0,0.0,1.0,15.0,12.0,8.0,15.0,11.0,8.0,13.0,18.0,11.0,7.0,1.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,22.0,20.0,12.0,25.0,13.0,10.0,19.0,12.0,3.0,11.0,0.0,9.0,0.0,0.0,0.0,0.0,D,2.05,3.25,4.00
6984,I1,15,2023/2024,2023-12-10,Inter,Udinese,3.0,1.0,2.0,2.0,0.0,1.0,0.0,1.0,11.0,8.0,21.0,10.0,13.0,13.0,7.0,9.0,4.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,3.0,3.0,1.0,0.0,12.0,6.0,13.0,12.0,16.0,15.0,13.0,14.0,5.0,1.0,7.0,5.0,0.0,0.0,0.0,0.0,D,1.22,6.44,2.67
6988,I1,15,2023/2024,2023-12-10,Salernitana,Bologna,0.0,2.0,2.0,0.0,3.0,1.0,2.0,2.0,11.0,13.0,9.0,10.0,12.0,11.0,14.0,15.0,2.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,0.0,11.0,13.0,11.0,6.0,18.0,16.0,15.0,12.0,3.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,D,4.50,3.45,1.85
6982,I1,15,2023/2024,2023-12-10,Empoli,Lecce,1.0,3.0,1.0,1.0,1.0,4.0,0.0,2.0,9.0,11.0,8.0,10.0,19.0,20.0,13.0,9.0,3.0,6.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,14.0,14.0,14.0,15.0,12.0,17.0,12.0,11.0,6.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,D,2.50,3.15,2.95
6981,I1,15,2023/2024,2023-12-10,Cagliari,Sassuolo,0.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,9.0,17.0,11.0,12.0,12.0,10.0,8.0,14.0,3.0,10.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,1.0,2.0,3.0,2.0,2.0,8.0,17.0,29.0,5.0,10.0,15.0,12.0,6.0,1.0,5.0,11.0,2.0,1.0,0.0,0.0,0.0,D,2.30,3.50,3.00
6980,I1,15,2023/2024,2023-12-10,Atalanta,Milan,0.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,10.0,16.0,8.0,14.0,14.0,15.0,13.0,13.0,3.0,5.0,7.0,6.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,0.0,1.0,0.0,2.0,1.0,10.0,9.0,10.0,14.0,11.0,9.0,16.0,13.0,6.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,D,2.49,3.18,2.92
6983,I1,15,2023/2024,2023-12-10,Frosinone,Torino,1.0,2.0,0.0,2.0,3.0,1.0,2.0,1.0,15.0,9.0,12.0,17.0,6.0,5.0,16.0,7.0,7.0,4.0,4.0,9.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,2.0,1.0,1.0,9.0,9.0,12.0,21.0,12.0,15.0,11.0,9.0,4.0,2.0,1.0,9.0,0.0,0.0,0.0,0.0,D,3.69,3.25,2.07
6989,I1,15,2023/2024,2023-12-10,Verona,Lazio,3.0,2.0,0.0,1.0,3.0,2.0,1.0,3.0,13.0,19.0,7.0,16.0,13.0,15.0,16.0,6.0,3.0,4.0,7.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,10.0,13.0,7.0,11.0,11.0,12.0,12.0,8.0,4.0,9.0,4.0,6.0,0.0,0.0,0.0,0.0,D,4.34,3.39,1.87
